# Survival Analysis Sandbox - WHAS500

In [9]:
# Core libraries
import pandas as pd 
import numpy as np 
import seaborn as sb 
import matplotlib.pyplot as plt

# Survival analysis and datasets 
import lifelines 
import sklearn
import sksurv 
from SurvSet.data import SurvLoader 

# Other analysis
from tableone import TableOne

### Load in datasets 

In [12]:
loader = SurvLoader()
whas500, ref = loader.load_dataset(ds_name='whas500').values()
whas500.head()

### Descriptives of dataset